## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [1]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from urllib.parse import quote_plus
import os

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [2]:
token = os.environ.get("ClickUpToken")

In [3]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

{'id': 8867829, 'username': 'Bogdan Pilyavets', 'email': 'rupilbo@yandex-team.ru', 'color': '#795548', 'profilePicture': None, 'initials': 'BP', 'week_start_day': None, 'global_font_support': False, 'timezone': 'Europe/Moscow'}


In [4]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [5]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': True}, 'time_tracking': {'enabled': True, 'harvest': False, 'rollup': False}, 'points': {'enabled': True}, 'custom_items': {'enabled':

In [6]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [7]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [8]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true
page=16&list_ids[]=44610695&include_closed=true
page=17&list_ids[]=44610695&include_closed=true
page=18&list_ids[]=44610695&include_closed=true
page=19&list_ids[]=44610695&include_closed=true


In [9]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def normalize_wrapper(data:List[dict], **kwargs) -> pd.DataFrame:
    """
    replaces the empty list with a dummy dictionary
    """
    for i in data:
        if not i['tags']:
            i['tags']=[{'name': "fake"}]  #TODO параметризовать ключи
    return pd.json_normalize(data, **kwargs)

def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(normalize_wrapper(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [37]:
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
#в последней странице почему-то остается только словарь, сейчас нет времени разбираться
missing_list = par_names.loc[par_names['standard_tags']=={'name': 'fake'}]     
par_names.loc[par_names['standard_tags']=={'name': 'fake'}, 'standard_tags'] = pd.Series([[{'name': 'fake'}]]*len(missing_list))

In [38]:
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names.loc[
    par_names['name'].isin([
        'Источник трафика',
        'Канал привлечения',
        'причина отказа',
        'Дата заявки',
        'campaign',
        'content/adset',
        'medium (cpc|cpm)',
        'source (UTM CRM)']), 
        ['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']
        ]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
par_names = par_names.rename(columns={'campaign': 'utm_campaign', 'medium (cpc|cpm)':'utm_medium', 'content/adset': 'utm_content', 'source (UTM CRM)': 'utm_source'})
missing_utms = par_names[par_names['utm_campaign'].isna()]


In [39]:
#разберем строку с utm-метками и превратим ее в 3 столбца
missing_utms['Источник трафика'] = missing_utms['Источник трафика'].fillna("")
missing_utms['utms'] = missing_utms['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
missing_utms['keys'] = missing_utms['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = missing_utms['keys'].astype(bool)
antimask = missing_utms['keys'].astype(bool) == False
utms = missing_utms.loc[mask, ('utms', 'keys')]
no_utms = missing_utms.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
utm_keys = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']
full_utms = full_utms[utm_keys]
#par_names = pd.concat([par_names, full_utms], axis=1)

<ipython-input-39-c46855cef850>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_utms['Источник трафика'] = missing_utms['Источник трафика'].fillna("")
<ipython-input-39-c46855cef850>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_utms['utms'] = missing_utms['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
<ipython-input-39-c46855cef850>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [40]:
#оставим только релевантные столбцы и выкинем каретки
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_medium', 'utm_content', 'причина отказа', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date
#почистим utm от лишних пробелов и приведем к нижнему регистру
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.strip())
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.lower())

In [41]:
par_names.sample(10)

name,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,причина отказа,Дата заявки
884,Ксения_счастливая_,u9f410,facebook,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,NaN,2021-08-13
471,Дарья Прибыткова (FB leads via Zapier),x18fww,NaN,yandex-math_test-offer_leadgen,cpc,NaN,0,2021-06-03
512,Евгения,pqc05t,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,NaN,2021-07-26
1397,Ольга Шварова,185tuhw,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,11,2021-07-22
1660,Юлия,h56zbw,facebook,NaN,NaN,NaN,0,2021-04-13
147,Алекса,6dv5yg,facebook,facebook_leads,NaN,facebook_leads,11,2021-02-06
69,Larisa Selivanova,1b3171t,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,15,2021-08-04
371,Валерия Константинова (FB leads via Zapier),z3c8j5,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,0,2021-06-26
400,Верун,znctvk,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,1,2021-07-03
1478,Светлана Долматова,u3yxx5,facebook,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,NaN,2021-08-13


In [15]:
#сформируем список тестовых айдишников
tags['standard_tags'] = tags['standard_tags'].astype(str)
throwaway=tags.loc[tags['standard_tags'].str.contains('тестовый|технический'), 'standard_id'].unique()

In [16]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
history = test_class.get_time_in_status(task_ids)

In [17]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,119e7p5,0. новый лид,#d3d3d3,open,0,604,1625603197254,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119eteq,0. новый лид,#d3d3d3,open,0,566,1625606086585,2.перезвонить на неделе,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119fjhx,0. новый лид,#d3d3d3,open,0,510,1625609723280,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119g3bp,0. новый лид,#d3d3d3,open,0,425,1625615578207,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119gcy3,0. новый лид,#d3d3d3,open,0,366,1626273866613,1. недозвон (нет контакта),#800000,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open
119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open
119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open
119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open
119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open


In [19]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true


In [20]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'value', 'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'type_config.options', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [21]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,value,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
404,b261ecb0-58b9-4a4e-bcae-1e1cafb9562f,Цель ученика и сроки достижения цели,short_text,1611823651718,False,False,NaN,NaN,NaN,NaN,...,Тимофей Выхристюк,\n,1623428054678,1629188596314,None,не планирует покупку,custom,evelinarap@yandex-team.ru,1628643600000,None
719,97b1b7e0-eccd-4a92-abfc-829147a4cbc5,первое занятие,date,1613757223046,False,False,NaN,NaN,NaN,NaN,...,Екатерина Тарасова,\n,1627201385419,1628499208264,None,возврат,custom,s-dayana21@yandex-team.ru,None,None
477,35dec42d-8229-417d-871a-1abf97209c77,класс,short_text,1622121639374,False,False,NaN,NaN,NaN,NaN,...,Ксения Лукина,\n,1627905872133,1627993042760,None,первая покупка,custom,rahmatulina@yandex-team.ru,None,None
242,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,NaN,NaN,NaN,...,Жанна Маточенко (Острогорская),2007 год рождения\n,1628668540703,1629188709481,None,первая покупка,custom,k-perfileva@yandex-team.ru,None,None
145,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Илья Мишуков,\n,1628863514873,1628863548139,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None


In [22]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,dzxcyu,Дмитрий Котов,\n,1629265012137,1629265040779,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
1,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,dzxcyu,Дмитрий Котов,\n,1629265012137,1629265040779,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
2,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,dzxcyu,Дмитрий Котов,\n,1629265012137,1629265040779,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
3,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'xba8ub', 'name': 'Татьяна Котова', 's...",dzxcyu,Дмитрий Котов,\n,1629265012137,1629265040779,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
4,e6cb4ac4-e138-4895-968d-9ebccaa12c78,Согласие на видеозапись,labels,1613132651997,False,False,NaN,dzxcyu,Дмитрий Котов,\n,1629265012137,1629265040779,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None


In [23]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Инциденты,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,
6fynn4,[],NaN,Дарья Карпухина,NaN,11\02 перерыв с 22\02 до 28\02,"чт 18-00 - 60 мин, пн 20-00 - 15 мин","{'id': '69yhrv', 'name': 'Татьяна Рушечникова ...",[1171db5a-754c-4178-85f1-67847ab15f03],"Дочь 6 класс, не очень любит математику, после...",NaN,"[{'id': 'ejzw6t', 'name': 'Варя Рушечникова па...",1613005200000
6fynn5,NaN,NaN,"Светлана Зайцева (15 мин), Сергей Христолюбов ...",NaN,16\02,"вт 16-00 60 мин, чт 16-00 15 мин","{'id': '6fu8tt', 'name': 'Анна Гуляева', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6w', 'name': 'Иван Гуляев пакет 4...",1613437200000
6fyppm,NaN,NaN,"Светлана Зайцева (15 мин), Анна И (60 мин)",NaN,NaN,"вскр 14-00 - 60 мин, вт 16-00 - 15 мин","{'id': 'd6zdz6', 'name': 'Ольга', 'status': '9...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'fk0rm6', 'name': 'Даша пакет 4+4 купл...",1612054800000
6hqz5e,[],NaN,Антон Загривин,"[{'id': 'rp4ytc', 'name': 'Аня Таран', 'status...",15\02,"пн 12-00 60 мин, чт 12-00 15 мин","{'id': '6dy7aj', 'name': 'Марина Таран', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6r', 'name': 'Аня пакет 4+4 купле...",1613350800000
6huakn,NaN,NaN,Света и Никита,NaN,27\02,"среда 17-00 60 мин, пятница 18-30 15 мин","{'id': '6fut3z', 'name': 'Наталья Кузнецова', ...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6c', 'name': 'Вася Кузнецов пакет...",1613523600000


In [24]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
6fynn4,69yhrv,Татьяна Рушечникова (Болгова)
6fynn5,6fu8tt,Анна Гуляева
6fyppm,d6zdz6,Ольга
6hqz5e,6dy7aj,Марина Таран
6huakn,6fut3z,Наталья Кузнецова


In [25]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']]
packages.head()

,purch_id
standard_id,
6fynn4,ejzw6t
6fynn4,71t1d4
6fynn4,7dyck9
6fynn4,gz8w39
6fynn4,gk7nub


In [26]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 305, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [27]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true
page=4&list_ids[]=48853634&include_closed=true


In [28]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df.head()

,value,standard_id,praktikum_id
3,https://praktikum-admin.yandex-team.ru/math/st...,w8d8bn,12149982
13,https://praktikum-admin.yandex-team.ru/math/st...,w8cy4h,7685273
23,https://praktikum-admin.yandex-team.ru/math/st...,w8chjw,7443072
33,https://praktikum-admin.yandex-team.ru/math/st...,w8c2r1,12145819
43,https://praktikum-admin.yandex-team.ru/math/st...,vhnhgw,7685273


In [29]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages.reset_index().merge(purch_df, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
6fynn4    7210563
6fynn5    7203187
6fyppm       None
6hqz5e    7245837
6huakn       None
Name: praktikum_id, dtype: object

In [30]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
6fyppm,d6zdz6,Ольга,None
nqbu1d,ztdune,Константин Макеев,10944231
pzgwq2,187w4e9,Наталья Прохорова,11693117
hk89z2,hh5wt5,Иван Богданов,7617804
ag1v19,pbbfwg,Евгения,11481318


In [31]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,причина отказа,Дата заявки
7236,z9f0x3,7. ждем оплаты,16.0,2021-07-02 20:26:37.189,custom,NaN,NaN,NaN,Ирада (Yandex-math.ru),z9f0x3,NaN,NaN,NaN,NaN,NaN,2021-06-29
1715,zfknhv,0. новый лид,11,2021-07-02 09:51:18.287,open,NaN,NaN,NaN,Зара,zfknhv,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,0,2021-07-02
2374,1aqv9b4,2.перезвонить на неделе,2813.0,2021-08-03 10:51:38.004,custom,NaN,NaN,NaN,Анастасия ЛИНОК,1aqv9b4,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,15,2021-08-03
3829,14ktny4,2.перезвонить на неделе,20092.0,2021-07-20 13:34:01.222,custom,NaN,NaN,NaN,Ольга Павловская,14ktny4,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,11,2021-07-15
7487,dx448u,1. нет первого контакта,28914.0,2021-03-11 16:28:20.121,custom,NaN,NaN,NaN,Светлана Мичкова,dx448u,facebook,facebook_ads,NaN,NaN,0,2021-02-07
4149,1b8xu5c,1. недозвон 2 сутки,1868.0,2021-08-07 12:23:47.289,custom,NaN,NaN,NaN,Рэйчел Якубович,1b8xu5c,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,0,2021-08-05
6967,p5exnb,1. недозвон (нет контакта),32876.0,2021-07-26 15:47:38.846,custom,NaN,NaN,NaN,Аминац,p5exnb,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,0,2021-07-16
2188,18braf8,2. отказ после контакта,36104.0,2021-07-24 09:58:36.622,custom,NaN,NaN,NaN,светлана бреднева,18braf8,NaN,yandex-math_test-offer_leadgen,cpc,int_child_everyone,15,2021-07-24
8215,6bqerr,отказ или отвал,146584.0,2021-03-11 16:33:12.830,closed,NaN,NaN,NaN,Светлана,6bqerr,facebook,facebook_ads,NaN,NaN,0,2021-02-01
7621,kq7e3w,9. есть оплата и запись,83526.0,2021-06-21 11:37:41.304,custom,kq7e3w,Дарья Горячева,9814868,Дарья Горячева,kq7e3w,NaN,NaN,NaN,NaN,NaN,2021-05-31


In [32]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,причина отказа,Дата заявки,этап,Созвон,Бронирование,Встреча
0,119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open,119e7p5,Светлана (Иван),10994925,Светлана (Иван),119e7p5,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,NaN,2021-07-06,0 заявка,0.0,0.0,0.0
1,119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open,NaN,NaN,NaN,Александра Борисова,119eteq,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,11,2021-07-07,0 заявка,0.0,0.0,0.0
2,119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open,NaN,NaN,NaN,КАТЕРИНА,119fjhx,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
3,119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open,119g3bp,Анна(Денис),11200929,Анна(Денис),119g3bp,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
4,119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open,NaN,NaN,NaN,Марина Белькова,119gcy3,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,0,2021-07-07,0 заявка,0.0,0.0,0.0
5,119gmeg,0. новый лид,303,2021-07-07 05:00:28.312,open,NaN,NaN,NaN,Светлана Керебко,119gmeg,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,15,2021-07-07,0 заявка,0.0,0.0,0.0
6,119gr05,0. новый лид,277,2021-07-07 05:28:11.314,open,NaN,NaN,NaN,Светлана Реутова,119gr05,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,0,2021-07-07,0 заявка,0.0,0.0,0.0
7,119h0p7,0. новый лид,174,2021-07-07 07:20:50.794,open,119h0p7,Галина Пеншина,11102355,Галина Пеншина,119h0p7,NaN,yandex-math_test-offer_leadgen,cpc,int_child_diagnostic,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
8,119h459,0. новый лид,139,2021-07-07 07:52:26.718,open,NaN,NaN,NaN,светлана иванова 888,119h459,NaN,yandex-math_test-offer_leadgen,cpc,leto_int_child,0,2021-07-07,0 заявка,0.0,0.0,0.0
9,119h57w,0. новый лид,128,2021-07-20 19:34:56.653,open,NaN,NaN,NaN,Светлана Андреевская,119h57w,NaN,yandex-math_test-offer_leadgen,cpc,int_child_diagnostic,NaN,2021-07-07,0 заявка,0.0,0.0,0.0


In [33]:
#создадим отдельные столбцы с датой созвона, брони и встречи
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
#выкинем лишние столбцы
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id']).rename(columns={'причина отказа': 'churn_reason'})
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
#конвертнем дату заявки в datetime, предварительно проверив на пустоту
if len(fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date']) > 0:
    fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,etap,call,meeting,booking,lead_date
7046,"8. есть оплата, нет записи",10131.0,2021-08-11 10:52:30.961,custom,12018395,Алина Острогорская,pzhxyw,facebook,yandex-math_leto_conv_traffic,cpc,int_child + yandex,NaN,6 онбординг,2021-08-06 14:29:27.810,2021-08-11 10:52:30.961,2021-08-08 17:33:20.506,2021-07-31
3116,2.перезвонить на неделе,5854.0,2021-08-12 16:13:09.837,custom,NaN,Елена Емейлл,u3rvpf,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,9,2 выясняем цели,2021-08-12 16:13:09.837,NaT,NaT,2021-08-12
4928,1. недозвон 2 сутки,5585.0,2021-06-17 15:24:14.677,custom,NaN,Ирина (FB leads via Zapier),x79fph,NaN,yandex-math_test-offer_leadgen,cpc,int_child_yellow,0,1 устанавливаем контакт,NaT,NaT,NaT,2021-06-05
2489,1. недозвон (нет контакта),83429.0,2021-06-21 13:14:29.227,custom,NaN,Оксана Литвак,83zx3z,NaN,NaN,NaN,NaN,0,1 устанавливаем контакт,NaT,NaT,NaT,2021-04-29
1487,0. новый лид,1229,2021-06-02 15:38:31.076,open,NaN,Мария Тахчиди (FB leads via Zapier),wxbpu5,NaN,yandex-math_test-offer_leadgen,cpc,NaN,0,0 заявка,NaT,NaT,NaT,2021-06-02
5296,1. недозвон (пингуем нед.),2407.0,2021-07-15 18:12:24.472,custom,NaN,светлана,13bme1t,NaN,yandex-math_leto_conv_traffic,NaN,int_child + yandex,0,1 устанавливаем контакт,2021-07-17 10:20:18.766,NaT,NaT,2021-07-13
8078,5.встреча подтверждена,65.0,2021-08-08 18:05:30.275,custom,NaN,Оксана,18c0dhj,NaN,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,NaN,4 подтверждаем встречу,2021-08-03 15:50:22.450,NaT,2021-08-04 19:21:34.725,2021-07-26


In [34]:
fin_df['standard_id'].nunique()

1824

In [35]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД